In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
results_folder = Path("results")

In [ ]:
def tensor_to_float(x):
    return float(x.replace("tensor(", "").replace(")", ""))

In [ ]:
methods = ["lrp", "gradient", "integrated_gradients", "smoothgrad"]
pretraining_methods = ["from_scratch", "pretrained"]
models = ["densenet", "swinvit", "vit"]
labels = [
    "Enlarged Cardiomediastinum",
    "Cardiomegaly",
    "Lung Opacity",
    "Lung Lesion",
    "Edema",
    "Consolidation",
    "Atelectasis",
    "Pneumothorax",
    "Pleural Effusion",
]
metrics = ["mass_accuracy", "rank_accuracy"]

In [ ]:
diff_results = []
for xai_method in methods:
    for pretraining in pretraining_methods:
        for label in labels:
            for model in models:
                pos_df = pd.read_csv(
                    results_folder
                    / f"finetuned_{model}"
                    / pretraining
                    / "normal_mask"
                    / f"_{label}_{xai_method}_output.csv"
                )
                neg_df = pd.read_csv(
                    results_folder
                    / f"finetuned_{model}"
                    / pretraining
                    / "inverse_mask"
                    / f"_{label}_{xai_method}_output.csv"
                )
                pos_df.mass_accuracy = pos_df.mass_accuracy.apply(
                    tensor_to_float
                )
                pos_df.rank_accuracy = pos_df.rank_accuracy.apply(
                    tensor_to_float
                )
                neg_df.mass_accuracy = neg_df.mass_accuracy.apply(
                    tensor_to_float
                )
                neg_df.rank_accuracy = neg_df.rank_accuracy.apply(
                    tensor_to_float
                )

                pos_df = pos_df.dropna()
                neg_df = neg_df.dropna()
                joined_df = pd.merge(
                    pos_df, neg_df, on="path", suffixes=("_pos", "_neg")
                )
                if joined_df.mass_accuracy_pos.isnull().sum() > 0:
                    print(pos_df)
                if joined_df.shape[0] == 0:
                    continue
                mass_accuracy_diff = (
                    joined_df.mass_accuracy_pos - joined_df.mass_accuracy_neg
                ).values
                rank_accuracy_diff = (
                    joined_df.rank_accuracy_pos - joined_df.rank_accuracy_neg
                ).values
                cur_df = pd.DataFrame(
                    {
                        "mass_accuracy_diff": mass_accuracy_diff,
                        "rank_accuracy_diff": rank_accuracy_diff,
                        "in_mask_mass_accuracy": joined_df.mass_accuracy_pos.tolist(),
                        "in_mask_rank_accuracy": joined_df.rank_accuracy_pos.tolist(),
                        "out_mask_mass_accuracy": joined_df.mass_accuracy_neg.tolist(),
                        "out_mask_rank_accuracy": joined_df.rank_accuracy_neg.tolist(),
                    }
                )
                cur_df["model"] = model
                cur_df["xai_method"] = xai_method
                cur_df["pretraining"] = pretraining
                cur_df["label"] = label
                diff_results.append(cur_df)

diff_df = pd.concat(diff_results, ignore_index=True, axis=0)

In [ ]:
diff_df[diff_df.xai_method == "lrp"].model.value_counts()

In [ ]:
diff_df.to_csv(results_folder / "metrics_diff_results.csv", index=False)